In [ ]:
# installing libraries that let pandas read parquet files

%pip install -U pyarrow fastparquet

In [ ]:
# importing the Path class so you can create Path objects directly
from pathlib import Path

In [ ]:
#importing pandas and reading the parquet file

import pandas as pd
df = pd.read_parquet("si_revwar.parquet")

In [ ]:
# checking table to confirm it loaded correctly
display(df)

In [ ]:
# checking what Python types are in the column indexed_object_types
df["indexed_object_types"].map(type).value_counts()


In [ ]:
# there are only NoneType values in the column, so I will drop those rows
df = df[df["indexed_object_types"].map(type) != type(None)]


In [ ]:
# the rest of the values are strings that look like JSON lists
# I want to parse them from strings to actual JSON lists
# (for example, "["note","voucher"]"" should become just ["note", "voucher"])
import json

def parse_indexed_object_types(cell):
    """
    converts a string that looks like a JSON list into a real JSON list
    """
    return json.loads(cell)          # parses original string into a JSON list

# creating new column with the parsed object types
df["object_types_list"] = df["indexed_object_types"].apply(parse_indexed_object_types)

In [ ]:
# checking what Python types are in the new column object_types_list now (they should all be lists)
df["object_types_list"].map(type).value_counts()

In [ ]:
# normalizing the lists in object_types_list

import re # for regular expressions
import unicodedata # for handling Unicode characters consistently

def normalize_token_basic(t):
    s = unicodedata.normalize("NFKC", str(t)) #transforms the string into a standardized Unicode form:
    s = s.lower().strip() #turns everything to lowercase and removes leading/trailing whitespace
    s = re.sub(r"[\u2010-\u2015\-_\/]+", " ", s) #replaces various dashes, underscores, and slashes with a single space
    return s #returns the cleaned-up string
df["object_types_norm"] = df["object_types_list"].apply(
    lambda items: [normalize_token_basic(x) for x in items]
)   # loops through each row in the column and passes the value in that row to the normalization function, applying it to each item in the lists


In [ ]:
# count unique full-cell values in column object_types_norm

# 1. converting each list into a string so pandas can compare easily
df["object_types_str"] = df["object_types_norm"].apply(str)

# 2. counting unique values and their frequencies
counts_df = (
    df["object_types_str"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "object_types", "object_types_str": "count"})
)

# 3. inspecting top values
display(counts_df.head(50))



In [ ]:
# there are a lot of variations of the same object types
# exporting the CSV to review the different object types and define a grouping strategy using an LLM
# canonicalization criteria provided to the LLM:
# 1. normalize spelling
# 2. merge synonyms under a single name (e.g., "pipe tampers", "pipe, tobaccos" → "pipes")
# 2. remove overly specific qualifiers (e.g., "silver spoon", "table spoon" → "spoons")
# 3. collapse minor variants into parent category (e.g., "dress, 1 piece", "dress fragments" → "dresses")
# 4. keep distinct when functionally differs (e.g., "box, stamps"  ≠ "box, doughs")
# 5. use plural form consistently (e.g., "painting" → "paintings")

counts_df.to_csv("unique_object_types_fullcells.csv")

In [ ]:
# loading and displaying the curated canonicalization mapping

mapping_df = pd.read_csv("canonical_map.csv")

# this CSV has two key columns:
# - "original_fullcell": the original unique string from the database
# - "canonical_plural": the standardized category, in plural form

display(mapping_df)

In [ ]:
# turning that csv into a dictionary

mapping_dict = pd.Series(mapping_df.canonical_plural.values, index=mapping_df.original_fullcell).to_dict()


In [ ]:
# applying the dictionary to the dataframe
# first, I need the column in the same format as "original_fullcell", which means
# converting the lists back to strings

df["object_types_str"] = df["object_types_norm"].astype(str)

In [ ]:
# mapping each row to its canonical category using the dictionary
# this function goes through every element in the column object_types_str from df, 
# looks them up in the mapping_dict dictionary and creates the new column canonical_type corresponding element form the  column from 

df["canonical_type"] = df["object_types_str"].map(mapping_dict)

In [ ]:
# checking how many rows successfully mapped

print("rows mapped:", df["canonical_type"].notna().sum())
print("rows unmapped:", df["canonical_type"].isna().sum())

In [ ]:
# displaying the df to see if the column was added correctly
display(df)

In [ ]:
# checking the first 50 rows with original and canonical categories side by side

df[["object_types_str", "canonical_type"]].head(50)

In [ ]:
# checking the frequency of the top 40 canonical type created

df["canonical_type"].value_counts().head(40)

In [ ]:
# exporting unique canonical categories with counts

# 1. counting frequency of each canonical category
counts = (
    df["canonical_type"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "canonical_type", "canonical_type": "count"})
)

# 2. inspecting the top categories
print(counts.head(20))

# 3. exporting to CSV for review, to define key verb families with a mixed strategy: 
# overall grouping using self-defined action families prompted to an LLM, followed by manual review and correction

counts.to_csv("canonical_types_counts.csv", index=False)

# proposed action families:
# 1. Commemorate & symbolize 
# 2. Decorate & furnish Dress & accessorize 
# 3. Eat, cook & drink 
# 4. Fight 
# 5. Heal & care 
# 6. Ignite & manage fire 
# 7. Measure & navigate 
# 8. Perform music 
# 9. Play 
# 10. Read, write & record 
# 11. Smoke 
# 12. Textile making 
# 13. Work & build 
# 14. Worship 
# 15. Other

In [ ]:
# loading and displaying the curated object - verb family mapping
# it has two key columns: canonical_type and action_family
map_df = pd.read_csv("object_verb_mapping.csv")
display(map_df)

In [ ]:
# creating df_with_action by taking the original df (that already has the canonical_type column) 
# and merging it with map_df, based on the canonical_type

df_with_action = df.merge(
    map_df,
    how="left"
)

# displaying df_with_actions
display(df_with_action)

In [ ]:
# checking if all elements have been assigned an action
print(df_with_action['action_family'].isnull().sum())


In [ ]:
df_with_action.to_csv("final_mapped_database.csv")
display(df_with_action)
# by reviewing the data, I notice that the action family defined as "Contain, store & preserve" contains mostly jars
# however, by looking at more details in other columns it is clear that these jars where used as pharmaceutical containers
# I re-tag the objects inside this action family to better match their purpose ("Heal & care")
# I also manually assign action families to other objects that were tagged as "Other".
# Importing the curated file with these changes and other manual re-tagging edits
df_with_action = pd.read_csv("final_mapped_database_manual.csv")


In [ ]:
# comparing the auto-mapped vs. manually-mapped CSVs to document every change made by hand,
# and exporting those differences as 'override' tables that can be reapplied later


# 1) input / output paths

# input
auto_file = Path("final_mapped_database.csv")
manual_file = Path("final_mapped_database_manual.csv")

# output
overrides_canonical_file = Path("manual_overrides_canonical.csv")
overrides_action_file = Path("manual_overrides_action.csv")
audit_rowlevel_file = Path("manual_overrides_rowlevel.csv")

# 2) read both CSVs
auto_df = pd.read_csv(auto_file)
manual_df = pd.read_csv(manual_file)

# 3) fixed join key and other important columns
id_col = "EDANurl"  # Smithsonian’s unique record URL
group_col = "object_types_str"  # original object-type label
fields_to_compare = ["canonical_type", "action_family"]

# 4) merge so each row shows auto + manual side by side
merged = auto_df[[id_col, group_col] + fields_to_compare] \
           .merge(
               manual_df[[id_col, group_col] + fields_to_compare],
               on=[id_col, group_col],
               suffixes=("_auto", "_manual"),
               how="inner"
           )

# 5) detect changed rows
changed_mask = False
for f in fields_to_compare:
    changed = merged[f"{f}_auto"].astype(str).fillna("") != \
              merged[f"{f}_manual"].astype(str).fillna("")
    changed_mask = changed if isinstance(changed_mask, bool) else (changed_mask | changed)

changed_rows = merged[changed_mask].copy()
changed_rows.to_csv(audit_rowlevel_file, index=False)

# 6) build one-to-one overrides by original label
def build_override(field):
    a, m = f"{field}_auto", f"{field}_manual"
    g = merged.groupby(group_col)[[a, m]] \
              .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else s.iloc[0]) \
              .reset_index().rename(columns={a: "auto", m: "manual"})
    return g[g["auto"].astype(str).fillna("") != g["manual"].astype(str).fillna("")]

ov_canonical = build_override("canonical_type")
ov_action    = build_override("action_family")

if not ov_canonical.empty:
    ov_canonical.rename(columns={group_col: "original_label",
                                 "manual": "canonical_type_manual"}, inplace=True)
    ov_canonical[["original_label", "canonical_type_manual"]] \
        .to_csv(overrides_canonical_file, index=False)

if not ov_action.empty:
    ov_action.rename(
        columns={
            group_col: "object_type",       # keep object type for reference
            "auto": "previous_action",   # old action
            "manual": "new_action"         # new action
        },
        inplace=True
    )
    ov_action.to_csv(overrides_action_file, index=False)

In [ ]:
# before creating the JSON structure from the final database to be used in the D3 treemap,
# I want to standardize the canonical_type values a bit more, since there are some categories that are very similar and will
# clutter the visualization. I will do this by merging some of the categories into broader ones, following these criteria:
# 1. create a table with all the unique canonical_type values and some other columns that will help decide how to group them)
# 2. use a mix of LLM prompting and manual review to define broader categories
# 3. create a mapping dictionary from the table and apply it to the final database

# This further canonization is done for display purposes.
# At this stage the action_family column has already been assigned using the
# original, more specific object types (e.g. "tinderboxes", "ballot boxes").
# Changing the canonical_type now will not affect the verbs/actions logic;
# it only merges similar objects (e.g. all bowls → "bowls") so the treemap
# labels are cleaner and easier to read.

# grouping by unique canonical_type and keeping one representative row for each type by adding some additional columns with details
unique_canon_df = ( 
    df.groupby("canonical_type", as_index=False)
    .first()[["canonical_type", "object_types_list", "objectType", "physicalDescription"]] 
    )

# saving the file to review and further canonization, where possible, with the support of an LLM
unique_canon_df.to_csv(Path("unique_canonical_types_for_review.csv"), index=False)

In [ ]:
# loading the necessary CSV files
df_with_action = pd.read_csv("final_mapped_database_manual.csv")
canon = pd.read_csv("unique_canonical_types_canonized.csv")

# build a mapping {original -> suggested}
mapping = dict(zip(canon["canonical_type"], canon["suggested_canonical"]))

df_canonized = df_with_action.copy()
df_canonized["canonical_type"] = (
    df_canonized["canonical_type"].map(mapping).fillna(df_canonized["canonical_type"])
)

df_canonized.to_csv("final_mapped_database_canonized.csv", index=False)

# final corrections
df_with_action_canonized = pd.read_csv("final_mapped_database_canonized_manualfix.csv")

In [ ]:
# replicating the same process as above to document the final manual fixes
# comparing the original vs manually-fixed CSVs to capture every change made by hand,
# and exporting those differences as "override" tables


# 1) input / output paths

# input
auto_file = Path("final_mapped_database_canonized.csv") # before final manual fixes
manual_file = Path("final_mapped_database_canonized_manualfix.csv") # after final manual fixes

# output
overrides_canonical_file = Path("manual_overrides_canonical_final.csv")
overrides_action_file = Path("manual_overrides_action_final.csv")
audit_rowlevel_file = Path("manual_overrides_rowlevel_final.csv")

# 2) read both CSVs
auto_df = pd.read_csv(auto_file)
manual_df = pd.read_csv(manual_file)

# 3) fixed join key and other important columns
id_col = "EDANurl"
group_col = "object_types_str"
fields_to_compare = ["canonical_type", "action_family"]

# 4) merge so each row shows auto + manual side by side
merged = auto_df[[id_col, group_col] + fields_to_compare] \
           .merge(
               manual_df[[id_col, group_col] + fields_to_compare],
               on=[id_col, group_col],
               suffixes=("_auto", "_manual"),
               how="inner"
           )

# 5) detect changed rows
changed_mask = False
for f in fields_to_compare:
    changed = merged[f"{f}_auto"].astype(str).fillna("") != \
              merged[f"{f}_manual"].astype(str).fillna("")
    changed_mask = changed if isinstance(changed_mask, bool) else (changed_mask | changed)

changed_rows = merged[changed_mask].copy()
changed_rows.to_csv(audit_rowlevel_file, index=False)

# 6) build one-to-one overrides by original label
def build_override(field):
    a, m = f"{field}_auto", f"{field}_manual"
    g = merged.groupby(group_col)[[a, m]] \
              .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else s.iloc[0]) \
              .reset_index().rename(columns={a: "auto", m: "manual"})
    return g[g["auto"].astype(str).fillna("") != g["manual"].astype(str).fillna("")]

ov_canonical = build_override("canonical_type")
ov_action    = build_override("action_family")

if not ov_canonical.empty:
    ov_canonical.rename(
        columns={
            group_col: "original_label",
            "manual": "canonical_type_manual"
        },
        inplace=True
    )
    ov_canonical[["original_label", "canonical_type_manual"]] \
        .to_csv(overrides_canonical_file, index=False)

if not ov_action.empty:
    ov_action.rename(
        columns={
            group_col: "object_type", # keep object type for reference
            "auto": "previous_action", # old action
            "manual": "new_action"     # new action
        },
        inplace=True
    )
    ov_action.to_csv(overrides_action_file, index=False)


In [ ]:
# creating the nested JSON structure for a D3 treemap

df_with_action_canonized = pd.read_csv("final_mapped_database_canonized_manualfix.csv")

# keeping only what is needed from the final database
base = (
    df_with_action_canonized[["action_family", "canonical_type"]]
)

# aggregating counts: one row per (action_family, canonical_type) with a count
#    - .size() counts how many records fall into each pair of keys
#    - renamig that "size" column to "count" for clarity
counts = (
    base
    .groupby(["action_family", "canonical_type"], as_index=False)
    .size()
    .rename(columns={"size": "count"})
)

# sorting action families in alphabetical order, then object types by descending count
counts = counts.sort_values(
    by=["action_family", "count", "canonical_type"],
    ascending=[True, False, True],
)

# building the nested structure that the D3 treemap expects:
#    {
#      "name": "root",
#      "children": [
#        {
#          "name": "<action_family>",
#          "children": [
#            { "name": "<canonical_type>", "value": <count> },
#            ...
#          ]
#        },
#        ...
#      ]
#    }

# the root is the top-level folder that contains everything (the database)
# each action_family is a child of the root
# each canonical_type is a child of the action_family
# each canonical_type has a value, which is its count (the treemap’s rectangles are sized by these values)

data = {"name": "root", "children": []}

# grouping the counts by action_family
for action_family, group_df in counts.groupby("action_family"):
    # for each canonical_type inside this action_family, make a leaf node
    children = [
        {
            "name": row["canonical_type"],
            "value": int(row["count"])
        }
        for _, row in group_df.iterrows()
    ]

    # adding this action_family (internal node) to the root’s children
    data["children"].append({
        "name": action_family,
        "children": children
    })

# saving the dictionary as a JSON file
# creating a path pointing to the output file
out_path = Path("treemap_data.json")
# converting the Python dictionary into a JSON string
out_path.write_text(
    json.dumps(data, ensure_ascii=False, indent=2), 
    encoding="utf-8"
    )


In [ ]:
# creating another JSON structure for the 3rd click level that maps each object type (column canonical_type)
# with its list of individual records (title, unitCode, collectionsURL)

#  this will be its shape:

#   {
#     "chair": [
#       {"title": "...", "unitCode": "...", "collectionsURL": "..."},
#       {"title": "...", "unitCode": "...", "collectionsURL": "..."}
#     ],
#     "spoon": [
#       {"title": "...", "unitCode": "...", "collectionsURL": "..."}
#     ]
#   }


# choosing the columns that will be used
col_object_type = "canonical_type" # this will be the grouping key
col_title = "title"           
col_unit = "unitCode"    
col_url = "collectionsURL"
col_identifier = "EDANurl"
col_action = "action_family" # this column will be used to filter the details panel not only by object type but also by action family

# preparing an empty dictionary to fill as the function iterates over rows
lookup = {}

# loop over every row in df_with_action
for _, row in df_with_action_canonized.iterrows():
    # get the object type for this row
    obj_type = row[col_object_type]
    # if the object type column is empty, skip it (just in case)
    if not obj_type:
        continue
    # create the small record we want to store
    item = {
        "title": row[col_title],
        "unitCode": row[col_unit],
        "collectionsURL": row[col_url],
        "EDANurl": row[col_identifier],
        "action_family": row[col_action],
    }
    # if the object type on the current row hasn’t appeared yet, create an empty list for it
    # this is to populate the lookup dictionary with every object type once (the first time it appears)     
    if obj_type not in lookup:
        lookup[obj_type] = []
    # and then append the record on the current row to the list for the current object type
    lookup[obj_type].append(item)

# saving the dictionary as a JSON file
# creating a path pointing to the output file
out_path = Path("object_details.json")
# converting the Python dictionary into a JSON string
out_path.write_text(
    json.dumps(lookup, ensure_ascii=False, indent=2), 
    encoding="utf-8"
    )


